## Portfolio Exercise: Starbucks
<br>

<img src="https://opj.ca/wp-content/uploads/2018/02/New-Starbucks-Logo-1200x969.jpg" width="200" height="200">
<br>
<br>
 
#### Background Information

The dataset you will be provided in this portfolio exercise was originally used as a take-home assignment provided by Starbucks for their job candidates. The data for this exercise consists of about 120,000 data points split in a 2:1 ratio among training and test files. In the experiment simulated by the data, an advertising promotion was tested to see if it would bring more customers to purchase a specific product priced at $10. Since it costs the company 0.15 to send out each promotion, it would be best to limit that promotion only to those that are most receptive to the promotion. Each data point includes one column indicating whether or not an individual was sent a promotion for the product, and one column indicating whether or not that individual eventually purchased that product. Each individual also has seven additional features associated with them, which are provided abstractly as V1-V7.

#### Optimization Strategy

Your task is to use the training data to understand what patterns in V1-V7 to indicate that a promotion should be provided to a user. Specifically, your goal is to maximize the following metrics:

* **Incremental Response Rate (IRR)** 

IRR depicts how many more customers purchased the product with the promotion, as compared to if they didn't receive the promotion. Mathematically, it's the ratio of the number of purchasers in the promotion group to the total number of customers in the purchasers group (_treatment_) minus the ratio of the number of purchasers in the non-promotional group to the total number of customers in the non-promotional group (_control_).

$$ IRR = \frac{purch_{treat}}{cust_{treat}} - \frac{purch_{ctrl}}{cust_{ctrl}} $$


* **Net Incremental Revenue (NIR)**

NIR depicts how much is made (or lost) by sending out the promotion. Mathematically, this is 10 times the total number of purchasers that received the promotion minus 0.15 times the number of promotions sent out, minus 10 times the number of purchasers who were not given the promotion.

$$ NIR = (10\cdot purch_{treat} - 0.15 \cdot cust_{treat}) - 10 \cdot purch_{ctrl}$$

For a full description of what Starbucks provides to candidates see the [instructions available here](https://drive.google.com/open?id=18klca9Sef1Rs6q8DW4l7o349r8B70qXM).

Below you can find the training data provided.  Explore the data and different optimization strategies.

#### How To Test Your Strategy?

When you feel like you have an optimization strategy, complete the `promotion_strategy` function to pass to the `test_results` function.  
From past data, we know there are four possible outomes:

Table of actual promotion vs. predicted promotion customers:  

<table>
<tr><th></th><th colspan = '2'>Actual</th></tr>
<tr><th>Predicted</th><th>Yes</th><th>No</th></tr>
<tr><th>Yes</th><td>I</td><td>II</td></tr>
<tr><th>No</th><td>III</td><td>IV</td></tr>
</table>

The metrics are only being compared for the individuals we predict should obtain the promotion – that is, quadrants I and II.  Since the first set of individuals that receive the promotion (in the training set) receive it randomly, we can expect that quadrants I and II will have approximately equivalent participants.  

Comparing quadrant I to II then gives an idea of how well your promotion strategy will work in the future. 

Get started by reading in the data below.  See how each variable or combination of variables along with a promotion influences the chance of purchasing.  When you feel like you have a strategy for who should receive a promotion, test your strategy against the test dataset used in the final `test_results` function.

In [16]:
# load in packages
from itertools import combinations

from test_results import test_results, score
import numpy as np
import pandas as pd

import scipy as sp
from scipy.stats import uniform

import sklearn as sk
from sklearn.preprocessing import PolynomialFeatures, StandardScaler, MinMaxScaler
from sklearn.linear_model import ElasticNet, LogisticRegression, LinearRegression
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.metrics import accuracy_score, classification_report
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier

import statsmodels.api as sm
import statsmodels.formula.api as smf

import matplotlib.pyplot as plt
import seaborn as sb
%matplotlib inline

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:80% !important; }</style>"))

In [17]:
# load in the data
train_data = pd.read_csv('./training.csv')
train_data.head()

,ID,Promotion,purchase,V1,V2,V3,V4,V5,V6,V7
0,1,No,0,2,30.443518,-1.165083,1,1,3,2
1,3,No,0,3,32.159350,-0.645617,2,3,2,2
2,4,No,0,2,30.431659,0.133583,1,1,4,2
3,5,No,0,0,26.588914,-0.212728,2,1,4,2
4,8,Yes,0,3,28.044332,-0.385883,1,1,2,2


In [18]:
def clean_data(data):
    df = data.copy()
    df['Promotion'] = df['Promotion'].replace(['Yes', 'No'], [1, 0]).astype(int)
    df = pd.get_dummies(df, columns=['V1', 'V4', 'V5', 'V6', 'V7'])
    return df


# Note that this function is no longer used
def add_features(df):
    
    # get covariates
    X = df.iloc[:,3:]
    
    # covariate polynomial and interaction terms
    poly = PolynomialFeatures(degree=3, interaction_only=False, include_bias=False).fit(X)
    X_poly = poly.transform(X)
    X_poly = pd.DataFrame(X_poly, columns=poly.get_feature_names())
    X_poly = X_poly.T.drop_duplicates().T

    # interaction with treatment
    promo_x_poly = X_poly.apply(lambda x: x*df['Promotion'])
    promo_x_poly.columns = ['promo_' + label for label in X_poly.columns]
    
    # full X df
    df_list = [df['Promotion'], promo_x_poly, X_poly]
    df_labels = ['Promotion'] + list(promo_x_poly.columns) + list(X_poly.columns)
    X = pd.concat(df_list, axis=1)
    X.columns = df_labels
    
    # drop remaining duplicates
    X = X.T.drop_duplicates().T
   
    y = df['purchase']
    
    return X, y

#### Experiment Results

In [7]:
## Experiment results
# ATE = ToT in our case
# ATE = 0.0095 and t = 12.529
# Note that ATE = IRR
exp_data = clean_data(train_data)
model = smf.ols(formula='purchase ~ 1 + Promotion + V2 + V3 + V1_0 + V1_1 + V1_2 + ' \
       'V1_3 + V4_1 + V4_2 + V5_1 + V5_2 + V5_3 + V5_4 + V6_1 + V6_2 + ' \
       'V6_3 + V6_4 + V7_1 + V7_2 + np.square(V2) + np.square(V3)', data=exp_data).fit()
ATE = model.params.Promotion
ATE_t = ATE / model.bse.Promotion
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:               purchase   R-squared:                       0.004
Model:                            OLS   Adj. R-squared:                  0.003
Method:                 Least Squares   F-statistic:                     19.02
Date:                Sun, 09 Jun 2019   Prob (F-statistic):           4.12e-55
Time:                        23:16:05   Log-Likelihood:                 66614.
No. Observations:               84534   AIC:                        -1.332e+05
Df Residuals:                   84517   BIC:                        -1.330e+05
Df Model:                          16                                         
Covariance Type:            nonrobust                                         
=================================================================================
                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------
Intercept         0.0082      0.003      2.353      0.019       0.001       0.015
Promotion         0.0095      0.001     12.491      0.000       0.008       0.011
V2               -0.0011      0.001     -1.647      0.099      -0.002       0.000
V3               -0.0008      0.000     -2.009      0.044      -0.002   -1.87e-05
V1_0              0.0030      0.001      2.434      0.015       0.001       0.005
V1_1              0.0024      0.001      2.226      0.026       0.000       0.004
V1_2              0.0028      0.001      2.658      0.008       0.001       0.005
V1_3          -5.919e-06      0.001     -0.005      0.996      -0.002       0.002
V4_1              0.0003      0.002      0.155      0.877      -0.003       0.004
V4_2              0.0079      0.002      4.444      0.000       0.004       0.011
V5_1              0.0032      0.001      2.716      0.007       0.001       0.006
V5_2             -0.0014      0.001     -1.317      0.188      -0.004       0.001
V5_3              0.0043      0.001      3.953      0.000       0.002       0.006
V5_4              0.0021      0.001      1.437      0.151      -0.001       0.005
V6_1              0.0022      0.001      2.055      0.040       0.000       0.004
V6_2              0.0020      0.001      1.818      0.069      -0.000       0.004
V6_3              0.0024      0.001      2.208      0.027       0.000       0.005
V6_4              0.0016      0.001      1.433      0.152      -0.001       0.004
V7_1              0.0042      0.002      2.344      0.019       0.001       0.008
V7_2              0.0040      0.002      2.234      0.025       0.000       0.007
np.square(V2)  1.694e-05   1.06e-05      1.598      0.110   -3.84e-06    3.77e-05
np.square(V3)    -0.0004      0.000     -0.851      0.395      -0.001       0.000
==============================================================================
Omnibus:                   119727.715   Durbin-Watson:                   2.000
Prob(Omnibus):                  0.000   Jarque-Bera (JB):         21302419.562
Skew:                           8.801   Prob(JB):                         0.00
Kurtosis:                      78.750   Cond. No.                     9.90e+18
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The smallest eigenvalue is 8.15e-28. This might indicate that there are
strong multicollinearity problems or that the design matrix is singular.
"""

In [8]:
# experiment NIR
# looks like a loss of $2,335, but this is a naive estimate
exp_data = clean_data(train_data)
NIR_data = exp_data[['Promotion', 'purchase']]
naive_NIR = 10 * NIR_data[NIR_data['Promotion']==1].purchase.sum() - 0.15 * NIR_data[NIR_data['Promotion']==1].Promotion.sum() - 10 * NIR_data[NIR_data['Promotion']==0].purchase.sum()
print('Naive NIR: ', naive_NIR)

# a better estimate would account for differences in covariates
# the result is the same - the "better" estimate is a loss of $2,349
better_NIR = 10 * ATE * NIR_data[NIR_data['Promotion']==1].shape[0] - 0.15 * NIR_data[NIR_data['Promotion']==1].Promotion.sum()
print('Better NIR: ', better_NIR)

Naive NIR:  -2334.6
Better NIR:  -2348.56486647


#### Strategy
I realize after sleeping on this problem that I was approaching it very wrong for awhile. Although I removed the old code, my first strategy was to estimate coefficients of interaction terms between promotion and variables. I was then going to use the coefficients/terms to estimate the effect of receiving the promotion at different values of the features. After many hours of grid search time, I had three different estimators with >98% accuracy and 0% recall. In hindsight, I should have used recall as the scoring parameter. 

Now I have a better idea that will be less computationally intensive and allow for arbitrary nonlinearity. I can just manually set the promotion variable to 1 and 0 for the test observations, and if the expected value of the observation's change in purchase likelihood is at least $0.15 then they should receive the promotion. This approach is sometimes called "preditive margins". It is important that my model maximize recall.

While I could make a customer scorer to maximize recall among the purchase=1 group, I am not doing that here simply because I am near the end of the Udacity Nanodegree program and I'd like to move forward.

In [67]:
# import and scale data
train_data = pd.read_csv('./training.csv')
df = clean_data(train_data)
X = df.drop(columns=['ID', 'purchase'])
y = df['purchase']

scaler = MinMaxScaler().fit(X)
X_scaled = scaler.transform(X)
X = pd.DataFrame(X_scaled, columns=X.columns)

In [10]:
ada_boost = AdaBoostClassifier(DecisionTreeClassifier(random_state=42), random_state=24)
ada_params = {'base_estimator__class_weight': [None, 'balanced'],
              'base_estimator__max_leaf_nodes': [3, 6, 8],
              'n_estimators': [100, 200, 300],
              'learning_rate': [1, 0.8, 0.6]
             }

In [53]:
#ada_cv = GridSearchCV(ada_boost, param_grid=ada_params, scoring='recall', verbose=2).fit(X, y)

Fitting 3 folds for each of 54 candidates, totalling 162 fits
[CV] base_estimator__class_weight=None, base_estimator__max_leaf_nodes=3, learning_rate=1, n_estimators=100 
[CV]  base_estimator__class_weight=None, base_estimator__max_leaf_nodes=3, learning_rate=1, n_estimators=100, total=   9.1s
[CV] base_estimator__class_weight=None, base_estimator__max_leaf_nodes=3, learning_rate=1, n_estimators=100 


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    9.9s remaining:    0.0s


[CV]  base_estimator__class_weight=None, base_estimator__max_leaf_nodes=3, learning_rate=1, n_estimators=100, total=   9.1s
[CV] base_estimator__class_weight=None, base_estimator__max_leaf_nodes=3, learning_rate=1, n_estimators=100 
[CV]  base_estimator__class_weight=None, base_estimator__max_leaf_nodes=3, learning_rate=1, n_estimators=100, total=   9.2s
[CV] base_estimator__class_weight=None, base_estimator__max_leaf_nodes=3, learning_rate=1, n_estimators=200 
[CV]  base_estimator__class_weight=None, base_estimator__max_leaf_nodes=3, learning_rate=1, n_estimators=200, total=  18.4s
[CV] base_estimator__class_weight=None, base_estimator__max_leaf_nodes=3, learning_rate=1, n_estimators=200 
[CV]  base_estimator__class_weight=None, base_estimator__max_leaf_nodes=3, learning_rate=1, n_estimators=200, total=  18.5s
[CV] base_estimator__class_weight=None, base_estimator__max_leaf_nodes=3, learning_rate=1, n_estimators=200 
[CV]  base_estimator__class_weight=None, base_estimator__max_leaf_no

[Parallel(n_jobs=1)]: Done 162 out of 162 | elapsed: 67.4min finished


In [11]:
ada_boost = AdaBoostClassifier(DecisionTreeClassifier(
    random_state=42, class_weight='balanced', max_leaf_nodes=3), random_state=24, learning_rate=0.6, n_estimators=100).fit(X, y)

In [13]:
ada_preds = ada_boost.predict(X)
print(f'AdaBoost accuracy = {accuracy_score(y, ada_preds)}')
print(classification_report(y, ada_preds))

AdaBoost accuracy = 0.7058698275250195
             precision    recall  f1-score   support

          0       0.99      0.71      0.83     83494
          1       0.03      0.67      0.05      1040

avg / total       0.98      0.71      0.82     84534



###### First grid search results: 0.67 recall among purchase=1 group
1. class_weight='balanced'
2. max_leaf_nodes=3
3. learning_rate=0.6
4. n_estimators=100

In [65]:
def promotion_strategy(df):
    '''
    INPUT 
    df - a dataframe with *only* the columns V1 - V7 (same as train_data)

    OUTPUT
    promotion_df - np.array with the values
                   'Yes' or 'No' related to whether or not an 
                   individual should recieve a promotion 
                   should be the length of df.shape[0]
                
    Ex:
    INPUT: df
    
    V1	V2	  V3	V4	V5	V6	V7
    2	30	-1.1	1	1	3	2
    3	32	-0.6	2	3	2	2
    2	30	0.13	1	1	4	2
    
    OUTPUT: promotion
    
    array(['Yes', 'Yes', 'No'])
    indicating the first two users would recieve the promotion and 
    the last should not.
    '''
    
    # get dummies
    X = pd.get_dummies(df, columns=['V1', 'V4', 'V5', 'V6', 'V7'])
    
    # scale data
    fake_col = pd.Series(np.ones(X.shape[0]), dtype=int)
    X_temp = pd.concat([fake_col, X], axis=1)
    X_scaled = scaler.transform(X_temp)
    X = pd.DataFrame(X_scaled[:, 1:], columns=X.columns)
    
    # create vectors of ones and zeroes
    zeros = pd.Series(np.zeros(X.shape[0]), dtype=int)
    ones = pd.Series(np.ones(X.shape[0]), dtype=int)
    
    # concatenate zeros and ones vectors to covariates
    X_nopromo = pd.concat([zeros, X], axis=1)
    X_nopromo.columns = ['promotion'] + list(X.columns)
    
    X_promo = pd.concat([ones, X], axis=1)
    X_promo.columns = ['promotion'] + list(X.columns)
 
    # get predictions with and without promotions
    pred_nopromo = ada_boost.predict(X_nopromo)
    pred_promo = ada_boost.predict(X_promo)
    
    result = []
    for nopromo, promo in zip(pred_nopromo, pred_promo):
        if promo - nopromo == 1:
            result.append('Yes')
        else:
            result.append('No')
            
    promotion = np.array(result)
    
    return promotion

In [66]:
# This will test your results, and provide you back some information 
# on how well your promotion_strategy will work in practice

test_results(promotion_strategy)

Nice job!  See how well your strategy worked on our test data below!

Your irr with this strategy is 0.0172.

Your nir with this strategy is 225.10.
We came up with a model with an irr of 0.0188 and an nir of 189.45 on the test set.

 How did you do?


(0.017183524040809316, 225.10000000000014)